In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('whitegrid')

In [35]:
#prices

dam = pd.read_csv('dam.csv')
fuels = pd.read_csv('fuels.csv')
cap = pd.read_csv('cap.csv')

In [43]:
dam = dam.set_index(pd.to_datetime(dam['Date']))
dam.columns = ['Date', 'nyca_dam', 'nyc_dam', 'li_dam', 'gj_dam']


In [37]:
fuels = fuels.set_index(pd.to_datetime(fuels['Date']))

In [46]:
cap = cap.set_index(pd.to_datetime(cap['Date'], format = '%b-%y'))
cap.columns = ['Date', 'nyca_icap', 'gj_icap', 'nyc_icap', 'li_icap']

In [39]:
locs = ['GJ', 'LI', 'NYC']

gen = {}

for loc in locs:
    gen[loc] = pd.read_csv(f'../data\Generating_Capacity_{loc}_11-23.csv')
    gen[loc] = (gen[loc]
                .set_index(pd.to_datetime(gen[loc]['Date']))
                .drop('Date', axis = 1))

gen['NYCA'] = pd.read_csv('../data\Generating_Capacity_NYCA_04-23.csv')
gen['NYCA'] = (gen['NYCA']
               .set_index(pd.to_datetime(gen['NYCA']['Date']))
               .drop('Date', axis = 1))